In [1]:
from langchain.agents import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from typing import Dict, List, Optional, TypedDict


In [2]:
import sys

# sys.path.append("../")

from custom_features.models import MODELS
from custom_features.params_setting_fct import set_api_keys, set_langsmith

set_api_keys()
set_langsmith()

In [17]:

# 1. Définition des Outils
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="Recherche sur Internet",
        func=search.run,
        description="utile pour lorsque vous avez besoin de répondre à des questions sur des sujets d'actualité. L'entrée doit être une requête de recherche.",
    ),
]


In [18]:

# 2. Définition des Agents (avec LLM et Prompt)
llm = ChatMistralAI(temperature=0)

prompt_chercheur = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Tu es un chercheur expert. Utilise l'outil de 'Recherche sur Internet' pour trouver des informations pertinentes pour répondre à la question de l'utilisateur. Une fois que tu as les informations, transmets-les à l'agent 'Répondeur'.",
        ),
        ("user", "{question}"),
    ]
)
prompt_repondeur = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Tu es un répondeur expert. On va te fournir des informations provenant d'un chercheur. Utilise ces informations pour formuler une réponse concise et informative à la question de l'utilisateur.",
        ),
        ("user", "Informations du chercheur: {informations}\n\nQuestion de l'utilisateur: {question}"),
    ]
)

# agent_chercheur = {"question": lambda state: state["question"]} | prompt_chercheur | llm
# agent_repondeur = {"informations": lambda state: state["informations"], "question": lambda state: state["question"]} | prompt_repondeur | llm
# Agent Chercheur MODIFIÉ pour retourner un dictionnaire mettant à jour l'état avec 'informations'
def agent_chercheur_node(state):
    question = state['question']
    resultat_recherche = search.run(question) # Utilisation de l'outil de recherche
    return {"informations": resultat_recherche} # Retourne un dictionnaire pour mettre à jour l'état

# Agent Répondeur MODIFIÉ (légèrement) pour accéder à 'informations' depuis l'état
def agent_repondeur_node(state):
    informations = state['informations']
    question = state['question']
    prompt_avec_informations = prompt_repondeur.format_messages(informations=informations, question=question)
    reponse = llm.invoke(prompt_avec_informations)
    return {"reponse": reponse.content} # Retourne la réponse, pourrait aussi mettre à jour l'état si nécessaire


In [20]:

# 3. Définition du Graph LangGraph
class AgentState(TypedDict):
    question: str
    informations: Optional[str]
    reponse: Optional[str] # 'reponse' sera mis à jour par le répondeur

builder = StateGraph(AgentState)

# Ajout des nodes (agents)
builder.add_node("chercheur", agent_chercheur_node)
builder.add_node("repondeur", agent_repondeur_node)

# Définition des edges (transitions)
builder.set_entry_point("chercheur")
builder.add_edge("chercheur", "repondeur")
builder.add_edge("repondeur", END) # Fin du graph après le répondeur

# Compilation du Graph
graph = builder.compile()


In [21]:

# 4. Exécution du Workflow
inputs = {"question": "Quelle est la capitale de la France?"}
resultats = graph.invoke(inputs)


In [22]:

# 5. Affichage du résultat
reponse_finale = resultats['informations'] # La réponse du répondeur est stockée dans 'informations' dans cet exemple simplifié.
print(f"Question: {inputs['question']}")
print(f"Réponse: {reponse_finale}")

Question: Quelle est la capitale de la France?
Réponse: Plongez dans le charme de Paris, la capitale de la France ! Explorez son histoire riche, ses attractions et sa cuisine délicieuse dans notre guide complet. Paris est la capitale de la France. Elle compte environ 2,2 millions d'habitants intra-muros et plus de 12 millions en intégrant les agglomérations alentours. Elle est située dans la partie nord de la France, est traversée par la Seine entre les confluents de la Marne et l'Oise. La capitale comptait au 1er janvier 2023 très précisément 2 133 111 habitants et ils sont répartis sur vingt arrondissements. Côté superficie, Paris s'étend sur 105,4 km². Et même si chaque année le nombre de parisiens tend à diminuer légèrement, Paris reste de loin la ville la plus peuplée de France. La capitale de la France est Paris. Située au nord du pays, Paris est la plus grande ville de France et l'une des plus influentes au monde. Elle est célèbre pour son riche patrimoine historique et culturel